In [1]:
import pandas as pd
import numpy as np
import pyspark

from pyspark.context import SparkContext

In [2]:
sc = SparkContext.getOrCreate()

23/02/21 15:27:11 WARN Utils: Your hostname, Nadias-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.10 instead (on interface en0)
23/02/21 15:27:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/21 15:27:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
data = range(1, 10001)

In [9]:
distData = sc.parallelize(data)

In [10]:
distData

PythonRDD[1] at RDD at PythonRDD.scala:53

RDD - resilient distributed dataset (RDD), which is a fault-tolerant collection of elements that can be operated on in parallel. There are two ways to create RDDs: parallelizing an existing collection in your driver program, or referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

In [14]:
# for spark-shell, doesn't work in Jupyter
#distData.filter(_ < 10).collect()
# returns Array[Int] = Array(1, 2, 3, 4, 5, 6, 7, 8, 9) (scala)

# for Jupyter Notebook, doesn't work in spark-shell
distData.filter(lambda x: x < 10).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [15]:
type(distData.filter(lambda x: x < 10).collect())

list

In [16]:
# prints the graph what needs to be calculated
distData.toDebugString()

b'(16) PythonRDD[1] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274 []'

In [18]:
dataTill10 = distData.filter(lambda x: x < 10)

In [19]:
dataTill10.toDebugString()

b'(16) PythonRDD[5] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274 []'

In [20]:
dataTill10.count()

9

In [21]:
distData.count()

10000

In [22]:
from operator import add

In [23]:
!ls

README.md                  spark-api.ipynb
Untitled.ipynb             spark.ipynb
deep_learning_lesson.ipynb spark101.py
mnist.pkl.gz               spark_step_by_step.ipynb
mnist_loader.py            spark_wrangle.ipynb
nn_from_scratch.py


#### Word count with Spark

In [24]:
f = sc.textFile('README.md')
wc = f.flatMap(lambda x: x.split(' ')).map(lambda x: (x, 1)).reduceByKey(add)
wc.saveAsTextFile('wc-out')

In [25]:
!ls

README.md                  spark-api.ipynb
Untitled.ipynb             spark.ipynb
deep_learning_lesson.ipynb spark101.py
mnist.pkl.gz               spark_step_by_step.ipynb
mnist_loader.py            spark_wrangle.ipynb
nn_from_scratch.py         wc-out


In [26]:
f

README.md MapPartitionsRDD[9] at textFile at NativeMethodAccessorImpl.java:0

In [27]:
wc

PythonRDD[17] at RDD at PythonRDD.scala:53

In [37]:
# to print debug string nicely use print and decode
print(wc.toDebugString().decode('utf=8'))

(2) PythonRDD[17] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[13] at mapPartitions at PythonRDD.scala:145 []
 |  ShuffledRDD[12] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[11] at reduceByKey at /var/folders/1h/vmdz8n3d00lfsp1hw8300jt00000gn/T/ipykernel_13710/2903505244.py:2 []
    |  PythonRDD[10] at reduceByKey at /var/folders/1h/vmdz8n3d00lfsp1hw8300jt00000gn/T/ipykernel_13710/2903505244.py:2 []
    |  README.md MapPartitionsRDD[9] at textFile at NativeMethodAccessorImpl.java:0 []
    |  README.md HadoopRDD[8] at textFile at NativeMethodAccessorImpl.java:0 []


In [38]:
wc.toDebugString

<bound method RDD.toDebugString of PythonRDD[17] at RDD at PythonRDD.scala:53>

Assignment:
- Use README and changes.txt
1. Create RDDs and filter each line by the word 'Spark'
2. Perform word count, so the results are (K,V) -> (word, count)
3. Join 2 RDDs

#### Spark API

First thing that Spark program does is create a `SparkContext` object, which tells Spark how to access a cluster. In the Scala or Python shell it is `sc` variable. Other programs must use a constructor to initiate a new `SparkContext`

The `master` parameter of SparkContext determines which cluster to use:
- `local`: local w/ 1 working thread
- `local[K]`: local with K working threads
- `spark://HOST:PORT` (default port 7077)
- `mesos://HOST:PORT` (default port 5050)